In [1]:
from depsurf import OUTPUT_PATH, Versions


results = []
for group in [Versions.ARCH, Versions.FLAVOR]:
    results.append(group.diff_pairs(result_path=OUTPUT_PATH / "config" / group.name))

[        utils.py:160] INFO: NumExpr defaulting to 8 threads.
[     versions.py:127] INFO: Diffing Arch
[     versions.py:134] INFO: Comparing x86 → arm to /Users/szhong/Downloads/bpf-study/output/config/Arch/5.4.0-26-generic-amd64_5.4.0-26-generic-arm64
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/5.4.0-26-generic-amd64.pkl
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/5.4.0-26-generic-arm64.pkl
[     versions.py:134] INFO: Comparing x86 → armhf to /Users/szhong/Downloads/bpf-study/output/config/Arch/5.4.0-26-generic-amd64_5.4.0-26-generic-armhf
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/5.4.0-26-generic-armhf.pkl
[     versions.py:134] INFO: Comparing x86 → ppc to /Users/szhong/Downloads/bpf-study/output/config/Arch/5.4.0-26-generic-amd64_5.4.0-26-generic-ppc64el
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-st

In [2]:
import pandas as pd
from depsurf.output import save_df

df = pd.concat(results, axis=1)
df = df.loc[(df != 0).any(axis=1)]
df = df.fillna(0).astype(int)
save_df(df, "config/summary")

[           pd.py:24 ] INFO: Saved dataframe to /Users/szhong/Downloads/bpf-study/output/config/summary.txt
[           pd.py:33 ] INFO: Saved dataframe to /Users/szhong/Downloads/bpf-study/output/config/summary.pkl


Arch                                           Flavor                                                             
                               x86 → arm x86 → armhf x86 → ppc x86 → s390x Generic → Lat. Generic → AWS Generic → GCP Generic → Oracle Generic → Azure
Function   Old                     48018       48018     48018       48018          48018         48018         48018            48018           48018
           New                     49362       48845     42826       33160          48034         46523         48149            49295           45483
           Added                    9216       12602      5444        3892             57           328           450             1594             992
           Removed                  7872       11775     10636       18750             41          1823           319              317            3527
           Changed                   120         106       137          78              0             2             1                1              10
           Param added                89          66       134          95              0             2             1                1              13
           Param removed              92          71       131         102              0             2             1                1              13
           Param reordered             3           8        12           8              0             0             0                0               2
           Param type changed         61          58        61          10              0             1             0                0               3
           Return type changed         9          10        24           7              0             0             0                0               1
Struct     Old                      8422        8422      8422        8422           8422          8422          8422             8422            8422
           New                      9075        8551      7368        6052           8425          8022          8367             8736            7846
           Added                    1659        1980       570         694              4            83            68              474             257
           Removed                  1006        1851      1624        3064              1           483           123              160             833
           Changed                   309        4077       379         375              7           132            14               17              32
           Field added               109         119       192         116              7             9             2                2              34
           Field removed             146         246       201         325              9            39            21                9             128
           Field type changed         87         165       148         119              0             1             2                1               8
Tracepoint Old                       752         752       752         752            752           752           752              752             752
           New                       685         690       648         559            752           747           752              761             739
           Added                      45          70        25          61              0             4             0                9              26
           Removed                   112         132       129         254              0             9             0                0              39
           Changed                     0         301         0          81              0             0             0                0               0
LSM        Old                       189         189       189         189            189           189           189              189             189
           New                       189         189       189         189            189           189       